## Housing Characteristics from the 2018 Census Planning Database: Tract Level
#### The PDB is a database of U.S. housing, demographic, socioeconomic and operational statistics based on select 2010 Decennial Census and select 5-year American Community Survey (ACS) estimates.   
#### Data are provided at the census tract level of geography.   
#### There are three tracts in Red Hook, Brooklyn. Tract 5300, 5900, 8500. 
#### These data can be used for many purposes, including survey field operations planning.  

In [1]:
__author__ = 'Linda Jaber, DRAWBrooklyn//CUSP, Spring_2018'

# imports for reading in, munging and calculating data
import pandas as pd
import json
import requests 
import urllib
import numpy as np
import io 

# reading in my api key saved in censusAPI.py as
# myAPI = 'XXXXXXXXXXXXXXX'
# request an api key in: https://api.census.gov/data/key_signup.html
from censusAPI import myAPI

# Python 3 compatibility
from __future__ import print_function, division

# Spatial
import geopandas as gpd
import fiona
import shapely

# Plotting
import matplotlib.pylab as pl
import seaborn as sns
sns.set_style('whitegrid')

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
# read in  the variables from 2018 Census Planning Database: Tract Level
url = "https://api.census.gov/data/2018/pdb/tract/variables.json"
resp = requests.request('GET', url)
PDB_18_tract = json.loads(resp.text)

In [3]:
# turning things into arrays to enable broadcasting
PDB_18_tract_keys = np.array(list(PDB_18_tract['variables'].keys()))

PDB_18_tract_keys

array(['pct_Single_Unit_ACS_12_16', 'pct_Diff_HU_1yr_Ago_ACS_12_16',
       'pct_ENG_VW_SPAN_ACSMOE_12_16', 'pct_Mailback_Count_CEN_2010',
       'Sngl_Prns_HHD_ACSMOE_12_16', 'Tot_Occp_Units_ACSMOE_12_16',
       'NH_White_alone_ACS_12_16', 'Renter_Occp_HU_ACS_12_16',
       'pct_Not_HS_Grad_ACSMOE_12_16', 'Civ_emp_16_24_ACSMOE_12_16',
       'Recent_Built_HU_ACS_12_16', 'pct_Prs_Blw_Pov_Lev_ACS_12_16',
       'pct_Mobile_Homes_ACS_12_16', 'MrdCple_Fmly_HHD_CEN_2010',
       'Civ_emp_16plus_ACS_12_16', 'pct_US_Cit_Nat_ACS_12_16',
       'TEA_Mail_Out_Mail_Back_CEN_2010', 'Age5p_German_ACSMOE_12_16',
       'MrdCple_Fmly_HHD_ACS_12_16', 'pct_Age5p_Russian_ACSMOE_12_16',
       'Civ_labor_45_64_ACS_12_16', 'Hispanic_CEN_2010',
       'pct_MrdCple_HHD_ACS_12_16', 'Tot_Prns_in_HHD_CEN_2010',
       'pct_One_Health_Ins_ACSMOE_12_16', 'State',
       'NH_Blk_alone_ACS_12_16', 'pct_Pop_5_17_CEN_2010',
       'Civ_unemp_16plus_ACSMOE_12_16', 'Mobile_Homes_ACSMOE_12_16',
       'pct_Not_MrdCpl

In [4]:
PDB_18_tract['variables']['Tot_Housing_Units_ACS_12_16']

{'concept': 'Housing Unit Variables',
 'group': 'N/A',
 'label': 'Total Housing Units in the ACS',
 'limit': 0,
 'predicateType': 'int'}

### Total Housing Units

In [5]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=Tot_Housing_Units_ACS_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
totalHU16 = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

totalHU16

,Tot_Housing_Units_ACS_12_16,state,county,tract,Unnamed: 4
0,1084,36,47,5300,NaN
1,676,36,47,5900,NaN
2,3089,36,47,8500,NaN


### Total Occupied Units 

In [6]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=Tot_Occp_Units_ACS_12_16,Tot_Occp_Units_CEN_2010,Tot_Occp_Units_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
Tot_Occp_Units = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

Tot_Occp_Units

,Tot_Occp_Units_ACS_12_16,Tot_Occp_Units_CEN_2010,Tot_Occp_Units_ACSMOE_12_16,state,county,tract,Unnamed: 6
0,982,844,52,36,47,5300,NaN
1,611,633,49,36,47,5900,NaN
2,3064,2833,57,36,47,8500,NaN


### Total Vacant Units 

In [7]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=Tot_Vacant_Units_CEN_2010,Tot_Vacant_Units_ACS_12_16,Tot_Vacant_Units_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
Tot_Vacant_Units = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

Tot_Vacant_Units

,Tot_Vacant_Units_CEN_2010,Tot_Vacant_Units_ACS_12_16,Tot_Vacant_Units_ACSMOE_12_16,state,county,tract,Unnamed: 6
0,144,102,41,36,47,5300,NaN
1,54,65,35,36,47,5900,NaN
2,17,25,40,36,47,8500,NaN


### Renter Occupied Units 

In [8]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=Renter_Occp_HU_CEN_2010,Renter_Occp_HU_ACS_12_16,Renter_Occp_HU_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
Renter_Occp_Units = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

Renter_Occp_Units

,Renter_Occp_HU_CEN_2010,Renter_Occp_HU_ACS_12_16,Renter_Occp_HU_ACSMOE_12_16,state,county,tract,Unnamed: 6
0,632,694,72,36,47,5300,NaN
1,565,549,53,36,47,5900,NaN
2,2819,3048,65,36,47,8500,NaN


### Owner Occupied Units 

In [9]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=Owner_Occp_HU_CEN_2010,Owner_Occp_HU_ACS_12_16,Owner_Occp_HU_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
Owner_Occp_Units = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

Owner_Occp_Units

,Owner_Occp_HU_CEN_2010,Owner_Occp_HU_ACS_12_16,Owner_Occp_HU_ACSMOE_12_16,state,county,tract,Unnamed: 6
0,212,288,60,36,47,5300,NaN
1,68,62,32,36,47,5900,NaN
2,14,16,26,36,47,8500,NaN


### Single Units 

In [10]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=Single_Unit_ACS_12_16,Single_Unit_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
Single_Units = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

Single_Units

,Single_Unit_ACS_12_16,Single_Unit_ACSMOE_12_16,state,county,tract,Unnamed: 5
0,225,63,36,47,5300,NaN
1,88,42,36,47,5900,NaN
2,0,16,36,47,8500,NaN


### Multi-Family Units // 2-9 Units

In [11]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=MLT_U2_9_STRC_ACS_12_16,MLT_U2_9_STRC_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
MLT_2_9 = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

MLT_2_9

,MLT_U2_9_STRC_ACS_12_16,MLT_U2_9_STRC_ACSMOE_12_16,state,county,tract,Unnamed: 5
0,683,98,36,47,5300,NaN
1,404,79,36,47,5900,NaN
2,189,110,36,47,8500,NaN


### Multi-Family Units // 10 or more Units

In [12]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=MLT_U10p_ACS_12_16,MLT_U10p_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
MLT_10p = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

MLT_10p

,MLT_U10p_ACS_12_16,MLT_U10p_ACSMOE_12_16,state,county,tract,Unnamed: 5
0,176,49,36,47,5300,NaN
1,184,50,36,47,5900,NaN
2,2900,215,36,47,8500,NaN


### Mobile Homes 

In [13]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=Mobile_Homes_ACS_12_16,Mobile_Homes_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
Mob = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

Mob

,Mobile_Homes_ACS_12_16,Mobile_Homes_ACSMOE_12_16,state,county,tract,Unnamed: 5
0,0,11,36,47,5300,NaN
1,0,11,36,47,5900,NaN
2,0,16,36,47,8500,NaN


### Crowded Units 
#### (Occupied Units with more than 1.01 persons per room in the ACS)

In [14]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=Crowd_Occp_U_ACS_12_16,Crowd_Occp_U_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
Crowd = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

Crowd

,Crowd_Occp_U_ACS_12_16,Crowd_Occp_U_ACSMOE_12_16,state,county,tract,Unnamed: 5
0,102,51,36,47,5300,NaN
1,49,30,36,47,5900,NaN
2,381,103,36,47,8500,NaN


### Recent Housing (built in 2010 or later)

In [15]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=Recent_Built_HU_ACS_12_16,Recent_Built_HU_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
Rcnt = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

Rcnt

,Recent_Built_HU_ACS_12_16,Recent_Built_HU_ACSMOE_12_16,state,county,tract,Unnamed: 5
0,5,7,36,47,5300,NaN
1,0,11,36,47,5900,NaN
2,0,16,36,47,8500,NaN


### Median Housing Value

In [16]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=Med_House_Value_ACS_12_16,Med_House_Value_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
Med_Value = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

Med_Value

,Med_House_Value_ACS_12_16,Med_House_Value_ACSMOE_12_16,state,county,tract,Unnamed: 5
0,"$897,200","$61,817",36,47,5300,NaN
1,"$856,500","$28,704",36,47,5900,NaN
2,null,null,36,47,8500,NaN


### Aggregate Housing Unit Value (dollars) 
#### (Sum of all value estimates for owner-occupied ACS housing units in an area)

In [17]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=Aggr_House_Value_ACS_12_16,Aggr_House_Value_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
Agg_Value = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

Agg_Value

,Aggr_House_Value_ACS_12_16,Aggr_House_Value_ACSMOE_12_16,state,county,tract,Unnamed: 5
0,"$262,627,000","$55,934,933",36,47,5300,NaN
1,"$47,140,000","$25,352,357",36,47,5900,NaN
2,null,null,36,47,8500,NaN


### Number of Households with Public Assistance Income

In [18]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=PUB_ASST_INC_ACS_12_16,PUB_ASST_INC_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
Pub_Asst = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

Pub_Asst

,PUB_ASST_INC_ACS_12_16,PUB_ASST_INC_ACSMOE_12_16,state,county,tract,Unnamed: 5
0,11,11,36,47,5300,NaN
1,17,16,36,47,5900,NaN
2,479,156,36,47,8500,NaN


### Percentage of Households with Public Assistance Income

In [19]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=pct_PUB_ASST_INC_ACS_12_16,pct_PUB_ASST_INC_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
Pct_Pub_Asst = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

Pct_Pub_Asst

,pct_PUB_ASST_INC_ACS_12_16,pct_PUB_ASST_INC_ACSMOE_12_16,state,county,tract,Unnamed: 5
0,1.12,1.12,36,47,5300,NaN
1,2.78,2.61,36,47,5900,NaN
2,15.63,5.08,36,47,8500,NaN


### Aggregate Households Income
#### Sum of all incomes in the household

In [20]:
url = 'https://api.census.gov/data/2018/pdb/tract?get=Aggregate_HH_INC_ACS_12_16,Aggregate_HH_INC_ACSMOE_12_16&for=tract:5300,5900,8500&in=state:36%20county:047&key=' + myAPI
response = requests.request('GET', url).content
Agg_HHD_Inc = pd.read_csv(io.StringIO(response.decode('utf-8').replace('[','').replace(']','')))

Agg_HHD_Inc

,Aggregate_HH_INC_ACS_12_16,Aggregate_HH_INC_ACSMOE_12_16,state,county,tract,Unnamed: 5
0,"$99,732,100","$10,354,819",36,47,5300,NaN
1,"$45,716,900","$5,895,533",36,47,5900,NaN
2,"$85,862,400","$27,834,948",36,47,8500,NaN
